In [ ]:
import numpy as np
import pandas as pd
import time
import re
# pip install -q -U google-generativeai
import google.generativeai as genai
import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold

In [10]:
print('VERSÕES')
print(f'Numpy: {np.__version__}')
print(f'Pandas: {pd.__version__}')
print(f'Genai: {genai.__version__}')
print(f'Scikit Learn: {sklearn.__version__}')

VERSÕES
Numpy: 2.3.5
Pandas: 2.3.3
Genai: 0.8.6
Scikit Learn: 1.7.2


In [ ]:
# GERADOR DE DADOS
def generate_poisson_data(n_samples=1000, complexity='simple'):
    x = np.linspace(0, 2, n_samples).reshape(-1, 1)
    if complexity == 'simple':
        y = -((x-1)**7)/7 - ((x-1)**3)/3
    else:
        y = np.sin(np.pi*x)**2
    return x, y.ravel()

In [ ]:
def load_api_key_from_txt(file_path="api_key.txt"):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            api_key = file.read().strip()
        return api_key
    except FileNotFoundError:
        print(f"Erro: Arquivo {file_path} não encontrado!")
        return None
    except Exception as e:
        print(f"Erro ao ler arquivo: {e}")
        return None

api_key = load_api_key_from_txt("chave.txt")

# CONFIGURAÇÃO DE ACESSO
GOOGLE_API_KEY = api_key 
genai.configure(api_key=GOOGLE_API_KEY)

# Seleção automática de modelo para evitar erro 404
try:
    available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
    model_id = next((m for m in available_models if 'gemini-1.5-flash' in m), available_models[0])
    model = genai.GenerativeModel(model_id)
    print(f"--> Modelo ativo para testes: {model_id}")
except Exception as e:
    print(f"Erro ao conectar na API: {e}")

--> Modelo ativo para testes: models/gemini-2.5-flash


In [13]:
def call_gemini_regression(x, case_type, examples=None):
    desc = "Poisson simples" if "Simples" in case_type else "Poisson oscilatória"
    prompt = f"Atue como um regressor matemático preciso.\nTarefa: Estimar u(x) para uma {desc}.\nx = {x:.4f}\n"
    if examples:
        prompt += "Exemplos base (x -> u(x)):\n" + "\n".join([f"{ex_x:.4f} -> {ex_y:.6f}" for ex_x, ex_y in examples])
    prompt += "\nResponda APENAS o número decimal (use ponto)."

    for attempt in range(3):
        try:
            response = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=0.0))
            # Extração numérica robusta com regex
            match = re.search(r"[-+]?\d*\.\d+|\d+", response.text.strip().replace(',', '.'))
            return float(match.group()) if match else 0.0
        except Exception as e:
            if "429" in str(e):
                print(f"   [Quota] Limite atingido. Aguardando 30s...")
                time.sleep(30)
                continue
            return 0.0
    return 0.0

In [14]:
datasets = {
    "Poisson_Simples": generate_poisson_data(complexity='simple'),
    "Poisson_Oscilatorio": generate_poisson_data(complexity='complex')
}

In [ ]:
# LOOP EXPERIMENTAL COM CÁLCULO DE DESVIO PADRÃO
all_results = []

for name, (X, y) in datasets.items():
    print(f"\nIniciando Processamento: {name}")
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    train_idx, test_idx = next(kf.split(X))
    
    # Redução estratégica para 10 amostras
    X_test_sub = X[test_idx][:10]
    y_test_sub = y[test_idx][:10]
    few_shot_context = list(zip(X[train_idx][:5].flatten(), y[train_idx][:5]))

    for strategy in ["Zero-Shot", "Few-Shot"]:
        print(f"  Estratégia: {strategy}...", end=" ")
        preds = []
        context = few_shot_context if strategy == "Few-Shot" else None
        
        for val_x in X_test_sub.flatten():
            preds.append(call_gemini_regression(val_x, name, examples=context))
            time.sleep(5.0) # Delay para respeitar o Free Tier
        
        # --- CÁLCULO DAS MÉTRICAS COM VARIÂNCIA ---
        preds = np.array(preds)
        y_true = np.array(y_test_sub)
        
        # Erros individuais para cálculo do desvio padrão
        errors_abs = np.abs(y_true - preds)
        errors_sq  = (y_true - preds)**2
        
        all_results.append({
            "Dataset": name,
            "Modelo": f"Gemini ({strategy})",
            "MSE": np.mean(errors_sq),
            "MSE stdev": np.std(errors_sq),
            "MAE": np.mean(errors_abs),
            "MAE stdev": np.std(errors_abs),
            "R2": r2_score(y_true, preds)
        })
        print("OK!")


Iniciando Processamento: Poisson_Simples
  Estratégia: Zero-Shot...    [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] Limite atingido. Aguardando 30s...
   [Quota] 

In [16]:
# 5. RESULTADO FINAL FORMATADO
df_gpt = pd.DataFrame(all_results)
print("\n" + "="*80)
print("TABELA COMPARATIVA FINAL - ETAPA 2 (LLM)")
print("="*80)
print(df_gpt[['Dataset', 'Modelo', 'MSE', 'MSE stdev', 'MAE', 'MAE stdev', 'R2']])


TABELA COMPARATIVA FINAL - ETAPA 2 (LLM)
               Dataset              Modelo       MSE  MSE stdev       MAE  \
0      Poisson_Simples  Gemini (Zero-Shot)  0.104612   0.036368  0.318875   
1      Poisson_Simples   Gemini (Few-Shot)  0.106322   0.039850  0.320875   
2  Poisson_Oscilatorio  Gemini (Zero-Shot)  0.014439   0.012014  0.102189   
3  Poisson_Oscilatorio   Gemini (Few-Shot)  0.014439   0.012014  0.102189   

   MAE stdev         R2  
0   0.054135 -30.116766  
1   0.057982 -30.625659  
2   0.063218  -2.612955  
3   0.063218  -2.612955  
